# Link Functions, Logistic Regressions, and Advanced AB Testing
**Current Status**: Code grade notebook

**Insert picture of a video game store**

Congratulations, in this lesson we are all data scientists at a video game shop. Our job is to help sell more video games. You have all sorts of data on customers and your boss wants to know who to market to. But before we get ahead of ourselves lets talk through the over vi

## Bayesian AB Test Review
* Revisit the basic AB test
* Estimate probability of a potential customer converting
* Interpret the result

## AB Testing Limitations
* Try AB Testing with a continuous observed value
  * Age of customer
* Understand why the "obvious" approaches fail

## Generalizing linear models with link functions
* Introduce the most important concept of this course  
  * Link functions
  * Generalized linear models
* (Re)introduce Transformations


## End to End "A Infinite Testing"
* Apply our new GLM 

## Logistic Regression in the real world
* Work through an example with a real world dataset

In [1]:
import bambi as bmb
from bambi import plots
import arviz as az
from scipy import stats
from scipy import special
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymc as pm
import seaborn as sns
import statsmodels


plt.style.use('intuitivebayes.mplstyle')

FIGSIZE = (16, 8)
rng = np.random.default_rng(0)

ModuleNotFoundError: No module named 'bambi'

# Hidden Section: Generate data
**Hello students**

Welcome to the hidden section where we generate the data. We've included this in the notebook purposefully so you can use it to aid your learnings. Simulation is an important tool because

* During learning you can create new scenarios to have an infinite amount of practice data
* During EDA you can use simulation to see what functional
* Check if your models are correctly specified, especially complex models, by simulating data and seeing if you can recover the parameters



In [ ]:
np.random.seed(SEED)

samples = 1000

# 1 indicates likes videogames
likes_videogames_bool = stats.bernoulli(p=.5).rvs(samples)
like_videogames_categorical = ["No" if x == 0 else "Yes" for x in likes_videogames_bool]

# Age
age = stats.randint(low=18, high=60).rvs(samples).astype(float)

df = pd.DataFrame({"age":age,
                   "likes_videogames":like_videogames_categorical,
                   "likes_videogames_bool":likes_videogames_bool}).sort_values("age")
df.head()

## Simulate our data
Unfortunately there are differing names for the sigmoid function in particular
* logistic (or logistic sigmoid)
* expit

Scipy uses the name expit. Since we use scipy in this course we'll use their naming so you learn both, and also can find the documentation for the code

In [ ]:
special.expit(3)

In [ ]:
# Parameters of model
intercept = 1
coeff_age = -0.1
coeff_likes_videogames = 2

# Construction of mean p_convert
y = intercept + coeff_age * df["age"] + coeff_likes_videogames * df["likes_videogames_bool"]

# Convert the continuous value to 0 1 bounded
# This is the link function
p_convert = special.expit(y)

# Simulate actual conversions. Rememver a prob of .99 doesn't guarantee someone will convert!
# To see that look at the graph below
converted_bool = stats.bernoulli(p=p_convert).rvs(random_state=rng)

df["p_convert"] = p_convert
df["converted_bool"] = converted_bool
df["converted"] = [{1:"Yes", 0:"No"}[outcome] for outcome in converted_bool]
df.head()

## Logistic curves

In [ ]:
fig, ax = plt.subplots(figsize=FIGSIZE)
sns.scatterplot(x="age", y="p_convert", hue="converted", data=df, ax=ax, s=200);

In [ ]:
customers_table = (df.drop("p_convert", axis=1).sample(frac=1, random_state=5).reset_index()
                          [["age", "likes_videogames", "converted", "likes_videogames_bool", "converted_bool"]])
customers_table.head(10)

## AB Testing at the video game store
**Insert picture of video game store

# Bayesian AB Test Review
What every company wants to know

## Your bosses as

"Take a look at our website checkout data.I want to know which customers are most likely to convert so we can focus our marketing efforts on them"

"I feel like age is important"

"Oh and also I want the results by the end of the day. I have a meeting with the CEO"

Your boss comes to you with a question, a question that basically every data scientist will get asked in their career. Given a new feature, or some properties about the customer, which are most likely to convert?

Now you're stressed out. You need to turn our results fast. Better get to it

## The data
* The age of the customer
* Whether they like video games or not
* Whether this customer has converted (or not)

In [ ]:
customers_table.head(10)

So we look at the data we have three things. The age, whether they like video, and whether they converted or not.

We can see four cases here
The first two are obvious.
We have
* People that don't like videogames that didn't convert
* People that did like videogmes and did convert

The second two though are more interesting
* Folks that didn't like videogames but did convert
* Folks that do like videogames that didn't convert


## Summarizing our DF

In [ ]:
(df
  .groupby("likes_videogames")
  .agg(
       converted = ("converted_bool", sum),
        visited = ("converted_bool", len)
    )
)

Because were short on time we start with the obvious thing, which is the boolean. Lets summarize our customer level data to 2x2 grid for simplicity. Just glancing at the rows we can already see a difference.

## Quick EDA

In [ ]:
fig, ax = plt.subplots(figsize=FIGSIZE)
ax.set_title("Conversions by like_videogame indicator")
ax.set_label("Count Conversion")
customers_table_summarized.plot(kind="bar", ax=ax);

We can plot the results as well  just as as double check

## AB Comparison in PyMC

In [ ]:
visited = customers_table_summarized["visited"].to_numpy()
converted = customers_table_summarized["converted"].to_numpy())
coords = {"age_buckets": customers_table_summarized.index}

with pm.Model(coords=coords) as ab_test:

    p_convert = pm.Beta("p_convert", 2, 2, dims="age_buckets")
    pm.Binomial("obs", p=p_convert, n=visited, observed=converted)

    idata_buckets = pm.sample()

We're not quite sure how to AB test in Bambi so for now we just use PyMC. Luckily the model is quite small so its quick to code up. Because we summed Everything here should look familiar to you


## Let's check out results

In [ ]:
az.plot_trace(idata_likes_video_games, kind="rank_bars");

Our sampling looks good, and we can already see from the KDE estimate that theres a big difference

## Comparing the two groups

In [ ]:
az.plot_forest(idata_likes_video_games);

Great, were now very sure that there's a difference, but this is very obvious. The subtle issue is age but lets discuss that in the next section

## Section Recap
* We want to know how conversion probability relates to
  * Whether people like video games
  * Age
* Coded a PPL model there is a difference in p_conversion on just the preference boolean

# Section 30: Regressing by age
* Your boss then says, well what about age?
  * I'm pretty sure older people are less likely to convert based on my experience
* we can see a trend and fit a line to it but that has problem
* Prob has to be between 0 or 1, most of these are invalid
* But what if we use this function here to squash the input
  * This is nothing new! You saw transformations in the last lesson

* The solution is a link function
  * This is most important part of this course
  * Now we're just formalizing it
* The link function we'll use here is sigmoid
  * Widely used in many fields like Neural nets etc

## How do include a continuous attribute into our framework?
* Our AB Testing framework was based on a discrete attribute (likes_videogame bool)
* How do we extend it?

## A, B, C, D, E, .... Comparison


In [ ]:
customers_table["age_buckets"] = pd.cut(customers_table["age"], bins=5)
customers_table

In [ ]:
customers_table_summarized = customers_table[["age_buckets", "converted_bool"]].groupby("age_buckets").agg([np.sum, len])
customers_table_summarized.columns = ["converted", "visited"]
customers_table_summarized

In [ ]:
customers_table_summarized = (customers_table.groupby("age_buckets")
    .agg(
        converted = ("converted_bool", sum),
        visited = ("converted_bool", len)
    )
)

## Another quick EDA

In [ ]:
fig, ax = plt.subplots(figsize=FIGSIZE)
ax.set_title("Conversions by like_videogame indicator")
ax.set_label("Count Conversion")
customers_table_summarized.plot(kind="bar", ax=ax);

## Our model

In [ ]:
coords = {"age_buckets": customers_table_summarized.index}

with pm.Model(coords=coords) as ab_test:
    p_convert = pm.Beta("p_convert", 2, 2, dims=("age_buckets"))
    
    obs = pm.Binomial("obs", p=p_convert,
                      n=customers_table_summarized["visited"].to_numpy(),
                      observed=customers_table_summarized["converted"].to_numpy())
    
    idata_buckets = pm.sample()

* Our AB testing framework only allowed for discrete variables
* Maybe we could bucket more

## Checking our inference

In [ ]:
az.plot_trace(idata_buckets);

Now we can see something going on, but this doesn't exactly answer our question of how age relates to conversion directly. We can compare group to group but our boss specifically wants to know exactly how the percentage conversion drops by age.

## But wait we know regression!

In [ ]:
converted_bool = df["converted_bool"].to_numpy()

with pm.Model() as ab_test:
    sigma = pm.Normal("p_convert", .001)
    β = pm.Normal("β", 0, .001, size=2)
    
    mu = pm.Deterministic("p", β[0] + β[1]*df["age"].to_numpy())
    
    # Switch to Bernoulli because we're predicting individual observations
    obs = pm.Bernoulli("obs", p=mu, observed=converted_bool)


If we take this to the logical extreme we could  plot conversion by age, but since conversion is just 0 or 1 we get this plot. How do we get a covariate for slope?

Note, potentially call this out as a difference between prior lessons to ensure its a learning point

## Lets try sampling

In [ ]:
with ab_test:
    idata_age_reg = pm.sample()

Ugh we're running into sampler issues. Looking at the error we see some things about negative p_convert but we don't ahve time for this

## Forget Bayes. Go back to OLS

In [ ]:
import statsmodels.formula.api as smf
results = smf.ols('converted_bool ~ age', data=df).fit()
results.params

We obviously see a trend related to age though and we want to use regression. Our bayesian regression isn't working so lets just go to statsmodel and use ordinary least squares. A

And look we immediately get an answer. The probability of conversin drop 1% for every year 
Check it out, now we know the prob of conversion across our observed vals. 

Lets go tell our boss!

## Issues when we look at our result

In [ ]:
fig, ax = plt.subplots(1)

converted_filter = df["converted_bool"].astype(bool)
# Add some jitter

jittered_y = df["converted_bool"] + stats.norm(scale=.05).rvs(df.shape[0])

alpha = .3
ax.scatter(df.loc[converted_filter, "age"],
          jittered_y[converted_filter],
          label="Converted",
          alpha=alpha)

ax.scatter( df.loc[~converted_filter, "age"],
           jittered_y[~converted_filter],
           label="Not Converted",
           alpha=alpha)


age_x = np.linspace(df["age"].min(), 80, 1000 )
y_pred = results.params[0] + results.params[1]*age_x
ax.plot(age_x, y_pred)
ax.legend()

But before we do lets just double check out plot. 
Negative probability? This is why our sampler was failing, it was telling us something
Oh man if we had reported these results we would have been fired
But look at the age of 60, something seems off?


## Section Recap
* Discrete AB testing is quite straightforward
* Run into issues when we're trying to estimate prob against a continuous variable
  * Discretizing isn't super usfeul
* Regression against average has
  * Non continuous
  * Probability can go over 0 and 1 which is impossible
* A big advantage of modern Bayesian frameworks is they "tell you" when something is wrong
  *  The standard solvers in most other tools don't tell you when something is wrong!
  * They just give you an answer

# Section 40: Link Functions


## Statistics and math review
* Probability is 0 1 bounded
* Linear regression is not

Linear regression will 
What if we can remap our observed to be between 0 and 1?


Turns out we you already know how. Remember our log transformation in the last lesson? What if we can apply a transformation?

## Sigmoid transformation
$$ \large
S(x) = \frac{1}{1 + e^{-x}} = \frac{e^x}{e^x + 1}
$$

Note this is also called
* Logistic 
* Expit 
* Logistic Sigmoid

Here's what it looks like in math

## Plotting the Sigmoid function
**Note** We will switch this to an animation to make it more clear what's happening

In [ ]:
x = np.linspace(-10, 10, 1000)
y = x
transformed_y = 1/(1+np.exp(-x))

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(25,8))
axes[0].plot(x, y)
axes[1].plot(x, transformed_y)

And here's what it looks like in code

## Logistic Regression in PyMC

In [ ]:
converted_bool = df["converted_bool"].to_numpy()

with pm.Model() as ab_test:
    
    # This is all the same linear regression as before
    sigma = pm.Normal("p_convert", 1)
    β = pm.Normal("β", 0, 1, size=2)
    
    mu = pm.Deterministic("mu", β[0] + β[1]*df["age"])
    
    # Our is our new sigmoid transoformation
    p = pm.Deterministic("p", pm.math.sigmoid(mu))
    
    # Switch to Bernoulli because we're predicting individual observations
    obs = pm.Bernoulli("obs", p=p, observed=converted_bool)
    idata_sigmoid = pm.sample()

You might notice we used the beta symbol here

In [ ]:
az.summary(idata_sigmoid, var_names=["β"])

In [ ]:
az.plot_trace(idata_sigmoid, var_names=["β"], compact=False, kind="rank_bars");

## This is a generalized linear model
$$
g(\mu_i) = \eta_i = \mathbf{x}_i^T\mathbf{\beta}
$$

We took an ordinary linear regression, and how we've generalized it. Its typically notated as this

G is the the **link function** in this case sigmoid. You're going to see other versions of this in the upcoming lesson

## Link functions in Bambi

In [ ]:
model = bmb.Model("converted[Yes] ~ age", df, family="bernoulli")
idata = model.fit(draws=2000, target_accept=0.85, random_seed=SEED)
az.summary(idata)

Turns out we can 
Look we get the same answer as the PyMC Model!

## Section Recap
* Transformations are the tool that turns linear regression into generalized linear equations
* Typically these are referred to as link functions 

# End to End Analysis with Bambi and A infinite testing
Lets go through the Bayesian workflow step by step to ensure we have a reliable result for out boss

## Single bool covariate regression

In [ ]:
model = bmb.Model("converted['Yes'] ~ likes_videogames+0", df, link="logit", family="bernoulli")
idata = model.fit(draws=2000, target_accept=0.85, random_seed=SEED)
az.summary(idata)

In [ ]:
special.expit(az.summary(idata)["mean"])

Lets start with the easiest thing. Now check this out. Do these numbers look familiar? that's because its the exact same thing as our first AB Test how cool is that? When we transform the means we end up with our original probabilities

## Lets fit on age and A infinite testing

In [ ]:
model = bmb.Model("converted['Yes'] ~ age", df, link="logit", family="bernoulli")
idata = model.fit(draws=2000, target_accept=0.85, random_seed=SEED)
az.summary(idata)

In [ ]:
fig, ax = plt.subplots(figsize=(9, 6))
plot_cap(model, idata, ["age"], ax=ax)
ax.set(ylabel="Conversion probability");

Check this out! We have the answer for our boss. As age increase prob decrease by this much.

This is basically A infinite testing because we have the p value as an INFINITE amount of ages along the x range. How freaking cool is that?

## Lets fit on age and likes videogames


In [ ]:
model = bmb.Model("converted['Yes'] ~ age + likes_videogames", df, link="logit", family="bernoulli")
idata = model.fit(draws=2000, target_accept=0.85, random_seed=SEED)
az.summary(idata)

In [ ]:
# # Testing an intercept only fit
# model_age_videogames = bmb.Model("converted['Yes'] ~ age + likes_videogames + 0", df, link="logit", family="bernoulli")
# idata_age_videogames = model_age_videogames.fit(draws=2000, target_accept=0.85, random_seed=SEED)
# az.summary(idata_age_videogames)

In [ ]:
fig, ax = plt.subplots(figsize=(9, 6))
plots.plot_cap(model_age_videogames, idata_age_videogames, ["age", "likes_videogames"], ax=ax);

Holy crap guys this result is truly mindblowing. Not only do we know the prob at every age. When we control for whether people like videogames or not we see a totally different trend.

Now when you go talk to your boss you not only provided the age insight, but you can recommend who to target per age and preference. And with this intuitive plot your boss is sure to give you a rise!

## Making a prediction

In [ ]:
df_predict = pd.DataFrame({"age": [20, 30, 40], "likes_videogames":["No","Yes", "Yes"]})
predictions = model.predict(idata, data=df_predict, inplace=False, kind="pps")

In [ ]:
predictions.posterior_predictive.mean(["chain", "draw"])["converted"]

Check this out
Turns out a 20 year old will convert at 20 percent rate, even if they dont like videogames, which is less than 30 year old that does like video games, and about the same as a 40 year old that likes videogames!

Now if any new customer comes in we can predict their prob conversion

## Section Recap
* AB testing is equivalent to categorical regression
* plot_cap helps us easily visualize continuous results
  * Gives us the answer for probability of conversion with age
* Since this is regression we can even control for preference quite easily
  * Can easily discern the differing trend in conversion between the two groups
* Predictions are quite easy as well!

# Logistic Regression in practice
**Insert applied example here**

The ANES is a nationally representative, cross-sectional survey used extensively in political science. This is a dataset from the 2016 pilot study, consisting of responses from 1200 voting-age U.S. citizens.

https://electionstudies.org/data-center/anes-2016-pilot-study/

In [ ]:
data = bmb.load_data("ANES")
data.shape

In [ ]:
data["vote"].value_counts()

In [ ]:
data = data.loc[data["vote"].isin(["clinton", "trump"]), :]
print(data.shape)
data.head()

In [ ]:
model_age = bmb.Model("vote[clinton] ~ party_id + scale(age)", data, family="bernoulli")
inf_data_age = model_age.fit(draws = 2000, tune = 2000)

In [ ]:
model_age.graph()

In [ ]:
az.summary(inf_data_age, var_names=["party_id", "scale(age)"])

## Create a new dataset with age going from 18 to 90

In [ ]:
age = np.arange(18, 91)
parties = ["democrat", "republican", "independent"]
new_data = pd.DataFrame({
    "age": np.tile(age, 3),
    "party_id": np.repeat(parties, len(age))
})
new_data

## Define Prediction Machine

In [ ]:
def make_vote_predictions(model, idata):

    # Predict mean of P(vote = clinton | age) by plugging in new data
    model.predict(idata, data=new_data)

    # Compute mean across chains
    posterior_mean = idata.posterior["vote_mean"].values.mean(0)

    # Select 500 draws
    # Transpose to get an array of shape (219, 500)
    posterior_mean = posterior_mean[:500, :].T
    return posterior_mean

posterior_mean = make_vote_predictions(model_age, inf_data_age)

In [ ]:
from matplotlib.lines import Line2D


def plot_vote_predictions(posterior_mean, ax):

    for i, party in enumerate(parties):
        idx = new_data.index[new_data["party_id"] == party]
        ax.plot(age, posterior_mean[idx], alpha=0.05, color=f"C{i}")

    ax.axhline(y = 0, ls="--", color="#a3a3a3", zorder=1)
    ax.axhline(y = 1, ls="--", color="#a3a3a3", zorder=1)
    ax.set_ylabel("P(vote=clinton | age)")
    ax.set_xlabel("Age", fontsize=15)
    ax.set_ylim(-0.025, 1.025)
    ax.set_xlim(18, 90)

    fig.subplots_adjust(top=0.925, left=0.1, right=0.95, bottom=0.1)

    labels = [party.capitalize() for party in parties]

    # Create legend
    handles = [
        Line2D(
            [0], [0], label=label, marker="o", color="None", 
            markerfacecolor=f"C{i}", markeredgewidth=0, markersize=10
        )
        for i, label in enumerate(labels)
    ]

    # Add legend
    x_center = 0.1 + (0.95 - 0.1) / 2 
    legend = fig.legend(
        handles=handles, 
        ncol=5,
        loc="lower center",
        bbox_to_anchor=(x_center, 0.925),
        handletextpad=0.2,
        columnspacing=2,
        fontsize=11,
    )

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))

posterior_mean = make_vote_predictions(model_age, inf_data_age)
plot_vote_predictions(posterior_mean, ax=ax)

## Voting Age Interaction Model

In [ ]:
model_age_interaction = bmb.Model("vote[clinton] ~ party_id + party_id:scale(age)", data, family="bernoulli")
idata_age_interaction = model_age_interaction.fit(draws = 2000, tune = 2000)

In [ ]:
model_age_interaction.graph()

In [ ]:
az.summary(idata_age_interaction, var_names=["party_id", "party_id:scale(age)"])

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))

posterior_mean = make_vote_predictions(model_age_interaction, idata_age_interaction)
plot_vote_predictions(posterior_mean, ax=ax)

## Section Recap
* Logistic Regression can help us understand voter preferences
* Interaction effects help us answer different questions
  * How does propensity to vote for one candidate or another?
  * How does it change per subgroup
* Plotting many posterior predictive samples helps us understand varability

# Lesson Recap


# Appendix Section: Binomial Model
Reviewer's note: This section would be recorded
If we'd rather prefer binomial heres how todo it.

## Original DataFrame at individual level

In [ ]:
df

## Aggregate to (Converted, Count)

In [ ]:
binom_df = df[["likes_videogames", "age", "converted_bool"]].groupby(["likes_videogames", "age"]).agg([np.sum, len])
binom_df.columns = ["Converted", "Count"]

binom_df  = binom_df.reset_index()
binom_df

In [ ]:
conversion_model = bmb.Model("p(Converted, Count) ~ likes_videogames + age", binom_df, family="binomial")
conversion_model_fitted = conversion_model.fit(draws=2000, target_accept=0.85, random_seed=SEED)

In [ ]:
az.summary(conversion_model_fitted)

In [ ]:
conversion_model = bmb.Model("p(Converted, Count) ~ likes_videogames + age", binom_df, family="binomial")
conversion_model_fitted = conversion_model.fit(draws=2000, target_accept=0.85, random_seed=SEED)